In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer
from xgboost.spark import SparkXGBClassifier
from pyspark.ml import PipelineModel
from pyspark.sql.types import *
from pyspark import keyword_only  ## < 2.0 -> pyspark.ml.util.keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param, Params, TypeConverters
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.ml.functions import vector_to_array
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql.functions import col, unix_timestamp, when
import pandas as pd
from time import time
# from miniapps.utils import nice_time
# from miniapps.utils import create_date_time_str
from pyspark.ml.feature import IndexToString, StringIndexer
import os
import warnings
from datetime import date
from pyspark.ml.classification import GBTClassifier 

In [2]:
from pyspark.ml.classification import GBTClassifier 

In [3]:
CONFIG_SPARK = [
        # ("spark.driver.memory", '4g'),
        # ("spark.driver.cores", '2'),
        ("spark.executor.memory", '5g'),
        ("spark.executor.cores", '3'),
        ("spark.executor.instances", '5')
    ]

In [4]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from time import time
import os

# os.environ['PYSPARK_PYTHON'] = './environment/bin/python'
s = time()
configs = CONFIG_SPARK
conf = SparkConf().setAll(configs)

spark = SparkSession \
    .builder \
    .appName("crosssla") \
    .config(conf=conf) \
    .getOrCreate()
print("done to get spark:")
print(spark)

23/12/17 21:55:43 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.1.27 instead (on interface eno1)
23/12/17 21:55:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/17 21:55:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/17 21:55:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/12/17 21:55:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/12/17 21:55:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


done to get spark:


In [5]:
data = spark.read.csv('data.csv', inferSchema=True, header=True)

In [6]:
data.columns

['CUSTOMER_ID',
 'COB',
 'FULL_NAME',
 'GENDER',
 'MARITAL_STATUS',
 'AGE',
 'MB_TENURE',
 'SECTOR',
 'EKYC_LEVEL',
 'CHANNEL',
 'ASSET_CLASS',
 'NBR_TXN_DAY',
 'NO_TXN_TM',
 'NBR_TXN_DAY_T_2',
 'NBR_TXN_DAY_T_3',
 'NBR_TXN_DAY_6M',
 'NBR_CREDIT_TXN_T_1',
 'NBR_DEBIT_TXN_T_1',
 'AMT_TOT_CREDIT_TXN_T_1',
 'AMT_TOT_DEBIT_TXN_T_1',
 'NBR_CREDIT_TXN_T_2',
 'NBR_DEBIT_TXN_T_2',
 'AMT_TOT_CREDIT_TXN_T_2',
 'AMT_TOT_DEBIT_TXN_T_2',
 'NBR_CREDIT_TXN_T_3',
 'NBR_DEBIT_TXN_T_3',
 'AMT_TOT_CREDIT_TXN_T_3',
 'AMT_TOT_DEBIT_TXN_T_3',
 'TOTAL_TXN_AMT_T_1',
 'TOTAL_NBR_TXN_T_1',
 'TOTAL_TXN_AMT_T_2',
 'TOTAL_NBR_TXN_T_2',
 'TOTAL_TXN_AMT_T_3',
 'TOTAL_NBR_TXN_T_3',
 'FIRST_DATE_OPEN_credit',
 'CONTRACT_COUNTS_credit',
 'INACTIVE_DAYS_credit',
 'TOTAL_TXN_NBR_T_1_credit',
 'TOTAL_TXN_NBR_T_2_credit',
 'TOTAL_TXN_NBR_T_3_credit',
 'TOTAL_TXN_NBR_6M_credit',
 'TOTAL_TXN_AMT_T_1_credit',
 'TOTAL_TXN_AMT_T_2_credit',
 'TOTAL_TXN_AMT_T_3_credit',
 'TOTAL_TXN_AMT_6M_credit',
 'FIRST_DATE_OPEN_DB',
 'CONTRAC

In [7]:
data.count()

563337

In [8]:
column_data_types = data.dtypes
columns = [data_type for column_name, data_type in column_data_types ]
print(set(columns))


{'int', 'date', 'string', 'double'}


In [8]:
LABEL_COL = 'pd_code'
LABEL_OUTPUT = LABEL_COL + '_'
FEAT = [
 'NBR_TXN_DAY',
 'NBR_TXN_DAY_T_2',
 'AMT_TOT_CREDIT_TXN_T_2'
]
data = data.fillna(value={**{x: 0.0 for x in FEAT}})

data = data.sample(0.01)



In [9]:
data.show()

23/12/17 21:56:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+----------+--------------------+------+--------------+---+------------+-------+----------+-------+-----------+-----------+---------+---------------+---------------+--------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------------+----------------------+--------------------+------------------------+------------------------+------------------------+-----------------------+------------------------+------------------------+------------------------+-----------------------+------------------+------------------+----------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+---------

In [10]:

labelIndexer = StringIndexer(inputCol=LABEL_COL, outputCol=LABEL_OUTPUT, handleInvalid='keep')
# indexer_df = labelIndexer.fit(data)
# data = indexer_df.transform(data)

assembler = VectorAssembler(inputCols=FEAT, outputCol="features")
# data = assembler.transform(data)

In [11]:
train, test = data.randomSplit([0.8, 0.2], seed=123)


In [12]:
params_setting = {'max_depth': 5,
                  'learning_rate': 0.03,
                  'num_classes': int(train.select(LABEL_COL).distinct().count()),
                  'num_workers' : 1}


In [13]:
from xgboost.spark import SparkXGBClassifier
from pyspark.ml import Pipeline 
xgb = SparkXGBClassifier(label_col=LABEL_OUTPUT, features_col="features", **params_setting)
stages = [labelIndexer, assembler, xgb]
pipeline = Pipeline(stages=stages)
ModelPipeline = pipeline.fit(train)
predTest = ModelPipeline.transform(test)

2023-12-17 21:57:26,561 INFO XGBoost-PySpark: _fit Running xgboost-2.0.2 on 1 workers with
	booster params: {'objective': 'multi:softprob', 'device': 'cpu', 'learning_rate': 0.03, 'max_depth': 5, 'num_classes': 87, 'num_class': 87, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}


In [20]:
predTest.show()

+-----------+-----------+--------------------+------+--------------+---+------------+-------+----------+-------+-----------+-----------+---------+---------------+---------------+--------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------------+----------------------+--------------------+------------------------+------------------------+------------------------+-----------------------+------------------------+------------------------+------------------------+-----------------------+------------------+------------------+----------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------

INFO:XGBoost-PySpark:Do the inference on the CPUs


In [21]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="pd_code_", predictionCol="prediction")
accuracy = evaluator.evaluate(predTest)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))
truePositive = evaluator.evaluate(predTest, {evaluator.metricName: "truePositiveRateByLabel", evaluator.metricLabel: 1.0})
print("truePositive: ", truePositive)
f1_score = evaluator.evaluate(predTest, {evaluator.metricName: "f1"})
print("f1 score:", f1_score)


2023-12-17 21:51:06,503 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-17 21:51:06,550 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-17 21:51:06,579 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-17 21:51:06,637 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-17 21:51:06,827 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-17 21:51:06,971 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


Accuracy = 0.9889705882352944
Test Error = 0.011029411764705621


2023-12-17 21:51:09,536 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-17 21:51:09,540 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-17 21:51:09,541 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-17 21:51:09,665 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-17 21:51:09,746 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-17 21:51:09,930 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


truePositive:  1.0


2023-12-17 21:51:12,494 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-17 21:51:12,495 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-17 21:51:12,591 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-17 21:51:12,611 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-17 21:51:12,622 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


f1 score: 0.9889705882352944


2023-12-17 21:51:12,704 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


In [22]:
# data.select(FEAT)

In [25]:
@udf
def orderByColValue(colValues):
    return [x[1] for x in sorted(zip(colValues, SORTED_LIST), key=lambda x: x[0], reverse=True)][:3]
SORTED_LIST = ModelPipeline.stages[0].labels
predTest = predTest.withColumn("product", orderByColValue('probability'))

In [24]:
# predTest.select(FEAT+['probability','product']).show(truncate=False)

In [26]:
predTest.show()

2023-12-17 21:53:07,349 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


+-----------+-----------+--------------------+------+--------------+---+------------+-------+----------+-------+-----------+-----------+---------+---------------+---------------+--------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------------+----------------------+--------------------+------------------------+------------------------+------------------------+-----------------------+------------------------+------------------------+------------------------+-----------------------+------------------+------------------+----------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------